In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stopwords/stopwords.txt
/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx


In [2]:
import matplotlib.pyplot as plt

In [3]:
file_path = '/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx'
df = pd.read_excel(file_path)

In [4]:
df.head()

,爬取时间(__time),爬取链接(__url),商品ID(product_id),评价时间(publish_time),评分（总分5分）(score),评价内容(content),评价者(author_name),评价者会员等级(author_level),商品sku(product_sku),评价标签(tags)
0,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550631798,5,此用户未填写评价内容,j***e,注册会员,单机版 小D黑色,[]
1,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633151,5,此用户未填写评价内容,c***n,钻石会员,单机版 小D黑色,[]
2,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633330,3,此用户未填写评价内容,j***1,银牌会员,单机版 小D黑色,[]
3,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633401,5,此用户未填写评价内容,苗***4,钻石会员,单机版 小D黑色,[]
4,2019-03-08 00:50:33,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633461,5,此用户未填写评价内容,J***3,注册会员,单机版 小D黑色,[]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71818 entries, 0 to 71817
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   爬取时间(__time)           71818 non-null  object
 1   爬取链接(__url)            71818 non-null  object
 2   商品ID(product_id)       71818 non-null  int64 
 3   评价时间(publish_time)     71818 non-null  int64 
 4   评分（总分5分）(score)        71818 non-null  int64 
 5   评价内容(content)          71818 non-null  object
 6   评价者(author_name)       71818 non-null  object
 7   评价者会员等级(author_level)  71818 non-null  object
 8   商品sku(product_sku)     68032 non-null  object
 9   评价标签(tags)             71809 non-null  object
dtypes: int64(3), object(7)
memory usage: 5.5+ MB


In [6]:
df['评价标签(tags)'].value_counts()

评价标签(tags)
[]                                     70214
["质量很好","面料不错"]                           71
["还可以"]                                   53
["速度快"]                                   44
["性价比高"]                                  38
                                       ...  
["效果不错","打印速度快"]                           1
["颜色鲜艳","没什么异味","挺好的东西"]                   1
["孩子喜欢","没什么异味","一般","价格给力","质量不错"]        1
["扭力大"]                                    1
["挺好的东西","积木数量多"]                          1
Name: count, Length: 725, dtype: int64

In [7]:
df = df[~df['评价内容(content)'].isin(['', ' ', '此用户未填写评价内容'])]
df = df.dropna(subset=['评价内容(content)'])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44422 entries, 0 to 44421
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   爬取时间(__time)           44422 non-null  object
 1   爬取链接(__url)            44422 non-null  object
 2   商品ID(product_id)       44422 non-null  int64 
 3   评价时间(publish_time)     44422 non-null  int64 
 4   评分（总分5分）(score)        44422 non-null  int64 
 5   评价内容(content)          44422 non-null  object
 6   评价者(author_name)       44422 non-null  object
 7   评价者会员等级(author_level)  44422 non-null  object
 8   商品sku(product_sku)     42054 non-null  object
 9   评价标签(tags)             44413 non-null  object
dtypes: int64(3), object(7)
memory usage: 3.4+ MB


In [8]:
df['评分（总分5分）(score)'].value_counts()

评分（总分5分）(score)
5    41832
1      960
4      852
3      528
2      250
Name: count, dtype: int64

In [9]:
neg_df = df[df['评分（总分5分）(score)'] <= 2].copy()
neg_df['label'] = 0

pos_df_full = df[df['评分（总分5分）(score)'] == 5].copy()
pos_df = pos_df_full.sample(n=len(neg_df), random_state=42)
pos_df['label'] = 1

df = pd.concat([neg_df, pos_df], ignore_index=True)
df = df[['评价内容(content)', 'label']].rename(columns={'评价内容(content)': 'content'})

df.head()

,content,label
0,一般般，一分钱一分货吧,0
1,您没有填写内容，默认好评,0
2,我买的是带遥感的的，结果就给我发了这个，我的手柄呢！？70块被你们白吃了，总是漏东西，催了n...,0
3,没用的东西。,0
4,重影，一般,0


In [10]:
import jieba
import re

with open('/kaggle/input/stopwords/stopwords.txt', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f])

def clean_text(text):
    text = str(text)
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)  
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and len(w) > 1]
    return ' '.join(words)

df['content'] = df['content'].apply(clean_text)

df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.844 seconds.
Prefix dict has been built successfully.


,content,label
0,一般般 一分钱 一分货,0
1,填写内容 默认 好评,0
2,遥感 给我发 手柄 白吃 东西 没见 补发 每次 帮忙 一点 反正 消费者 气死我了 一款 ...,0
3,没用 东西,0
4,重影,0


In [11]:
from sklearn.model_selection import train_test_split

texts = df['content'].tolist()
labels = df['label'].tolist()

X_temp, X_test, y_temp, y_test = train_test_split(texts, labels, test_size=0.1, random_state=42, stratify=labels)

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.111, random_state=42, stratify=y_temp)

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [13]:
class JDDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts  # list of cleaned text strings
        self.labels = labels  # list of 0 or 1

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'text': self.texts[idx],
            'label': int(self.labels[idx])
        }


In [16]:
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    labels = [item['label'] for item in batch]
    
    encoding = tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

    labels = torch.tensor(labels, dtype=torch.long)

    return {
        'input_ids': encoding['input_ids'],             
        'attention_mask': encoding['attention_mask'], 
        'label': torch.tensor(labels) 
    }

In [17]:
train_loader = DataLoader(JDDataset(X_train, y_train), batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(JDDataset(X_val, y_val), batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(JDDataset(X_test, y_test), batch_size=32, shuffle=False, collate_fn=collate_fn)


In [18]:
import torch.nn as nn
from transformers import BertModel

class BertForClassification(nn.Module):
    def __init__(self, freeze_bert=True):
        super(BertForClassification, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.classifier(x)
        return logits


2025-05-22 18:17:16.110946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747937836.133301     290 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747937836.140111     290 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
from torch.utils.tensorboard import SummaryWriter
import os
from tqdm import tqdm

def train(model, train_loader, val_loader, epochs, log_dir):
    writer = SummaryWriter(log_dir=log_dir)
    global_step = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            writer.add_scalar("Train/Loss", loss.item(), global_step)
            global_step += 1

        acc = correct / total
        writer.add_scalar("Train/Accuracy", acc, epoch)

        # 验证
        val_acc, val_loss = evaluate(model, val_loader)
        writer.add_scalar("Val/Loss", val_loss, epoch)
        writer.add_scalar("Val/Accuracy", val_acc, epoch)

        print(f"Epoch {epoch+1}, Train Acc: {acc:.4f}, Val Acc: {val_acc:.4f}")

    writer.close()

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total, total_loss / len(dataloader)


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()


In [21]:
# 冻结 BERT，只训练分类头
frozen_model = BertForClassification(freeze_bert=True).to(device)
optimizer = torch.optim.AdamW(frozen_model.parameters(), lr=2e-5)
train(frozen_model, train_loader, val_loader, epochs=20, log_dir="/kaggle/working/runs/bert_frozen")

# 保存冻结模型
torch.save(frozen_model.state_dict(), "/kaggle/working/bert_frozen.pth")


Epoch 1/20:   0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_290/3204579863.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.stack([torch.tensor(l) for l in labels])
Epoch 1/20: 100%|██████████| 61/61 [00:07<00:00,  8.26it/s]


Epoch 1, Train Acc: 0.4845, Val Acc: 0.4545


Epoch 2/20: 100%|██████████| 61/61 [00:07<00:00,  8.68it/s]


Epoch 2, Train Acc: 0.5010, Val Acc: 0.5041


Epoch 3/20: 100%|██████████| 61/61 [00:07<00:00,  8.67it/s]


...

Epoch 18, Train Acc: 0.6410, Val Acc: 0.6694


Epoch 19/20: 100%|██████████| 61/61 [00:07<00:00,  8.64it/s]


Epoch 19, Train Acc: 0.6100, Val Acc: 0.6860


Epoch 20/20: 100%|██████████| 61/61 [00:07<00:00,  8.59it/s]


Epoch 20, Train Acc: 0.6198, Val Acc: 0.7025


In [22]:
# 不冻结 BERT，微调模型
unfrozen_model = BertForClassification(freeze_bert=False).to(device)
optimizer = torch.optim.AdamW(unfrozen_model.parameters(), lr=2e-5)
train(unfrozen_model, train_loader, val_loader, epochs=20, log_dir="/kaggle/working/runs/bert_unfrozen")

# 保存不冻结模型
torch.save(unfrozen_model.state_dict(), "/kaggle/working/bert_unfrozen.pth")


Epoch 1/20:   0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_290/3204579863.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.stack([torch.tensor(l) for l in labels])
Epoch 1/20: 100%|██████████| 61/61 [00:23<00:00,  2.55it/s]


Epoch 1, Train Acc: 0.7913, Val Acc: 0.8636


Epoch 2/20: 100%|██████████| 61/61 [00:23<00:00,  2.55it/s]


Epoch 2, Train Acc: 0.8946, Val Acc: 0.8843


Epoch 3/20: 100%|██████████| 61/61 [00:23<00:00,  2.54it/s]


...

Epoch 18, Train Acc: 0.9747, Val Acc: 0.9091


Epoch 19/20: 100%|██████████| 61/61 [00:23<00:00,  2.54it/s]


Epoch 19, Train Acc: 0.9731, Val Acc: 0.9008


Epoch 20/20: 100%|██████████| 61/61 [00:24<00:00,  2.54it/s]


Epoch 20, Train Acc: 0.9401, Val Acc: 0.8843


In [32]:
import torch.nn.functional as F

def predict(model, texts, max_length=128):
    inputs = tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)

    # 模型预测
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    return preds.cpu().tolist(), probs.cpu().tolist()


In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

model = BertForClassification(freeze_bert=False).to(device)
model.load_state_dict(torch.load("/kaggle/working/bert_unfrozen.pth", map_location=device))
model.eval()  


BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [25]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# model = BertForClassification(freeze_bert=True).to(device)
# model.load_state_dict(torch.load("/kaggle/working/bert_frozen.pth", map_location=device))
# model.eval()  

In [26]:
texts = [
    "快递速度很快，商品包装很好，非常满意",
    "用了几天就坏了，质量太差了",
    "中规中矩，没有特别的感觉"
]

# 预测
preds, probs = predict(model, texts)

# 输出
for text, pred, prob in zip(texts, preds, probs):
    print(f"【评论】：{text}\n→ 预测标签：{pred}（{['负面','正面'][pred]}），概率：{prob}\n")


【评论】：快递速度很快，商品包装很好，非常满意
→ 预测标签：1（正面），概率：[0.00935341976583004, 0.9906466007232666]

【评论】：用了几天就坏了，质量太差了
→ 预测标签：0（负面），概率：[0.9913827776908875, 0.008617241866886616]

【评论】：中规中矩，没有特别的感觉
→ 预测标签：1（正面），概率：[0.029887964949011803, 0.9701120257377625]



In [ ]:
# tensorboard --logdir=runs